In [ ]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar
import time
import re

#Logger
logging.basicConfig(filename='Anàlisi-GetBotometerScore.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
##############
# PARAMETERS #
##############

client = MongoClient(username='XXX', password='XXX')

DatabaseName = "Streaming"
CollectionName = "Users"

db = client[DatabaseName]

In [ ]:
###################
# BOTOMETER LOGIN #
###################

import botometer

#Key from BOTOMETER API
mashape_key = "XXX"

#Dictionary with the credentials for the Twitter APIs
twitter_app_auth = {
    'access_token' : "XXX",
    'access_token_secret' : "XXX",
    'consumer_key' : "XXX",
    'consumer_secret' : "XXX",    
}

#Connecting to the botometer API
bom = botometer.Botometer(wait_on_ratelimit = True, mashape_key = mashape_key, **twitter_app_auth)

In [ ]:
####################################
# GET BOTOMETER INFO AND UPDATE DB #
####################################

ids = db[CollectionName].find({'botometer_score':{'$exists':False}}, {'_id':1})
print(ids.count())

MAX_API_CALLS = 17280
count = 0

for id in ids:
    try:
        result = bom.check_account(id["_id"])
        #print(result)
    except Exception as e:
        auth_match = re.search('Not authorized', str(e))
        timeline_match = re.search('has no tweets in timeline', str(e))
        notExist_match = re.search('Sorry, that page does not exist', str(e))
        overCapacity_match = re.search('Over capacity', str(e))
        
        if auth_match:
            result = "Not authorized"
        elif overCapacity_match:
            result = "Over Capacity"
        elif timeline_match:
            result = "Has no tweets in timeline"
        elif notExist_match:
            result = "Does not exists anymore"
        else:
            result = "Error"
            logging.error("Exception. Message:", e)
            logging.error("Sleeping for 1 min")
            time.sleep(60)
    
    try:
        db[CollectionName].update_one(
                                    {'_id': id['_id']},
                                    {'$set': {'botometer_score':result}},
                                    upsert=True
                                  )
    except Exception as e:
        logging.error(e)
        logging.error("Fatal exception inserting users in MongoDB")
'''        
    count +=1
    if(count>MAX_API_CALLS):
        logging.error("Daily rate limit! Sleeping for 1h")
        time.sleep(36000)
        count = 0
'''